In [ ]:
#| default_exp xml

# XML

> Concise generation of XML.

In [ ]:
#| export
from fastcore.utils import *

import types
from dataclasses import dataclass
from functools import partial
from html import escape

In [ ]:
from IPython.display import Markdown

In [ ]:
#| export
def _attrmap(o):
    return dict(cls='class', klass='class', fr='for').get(o, o)

In [ ]:
#| export
def xt(tag:str, *c, **kw):
    "Create an XML tag structure `[tag,children,attrs]` for `toxml()`"
    if len(c)==1 and isinstance(c[0], types.GeneratorType): c = tuple(c[0])
    kw = {_attrmap(k.lstrip('_')):str(v) for k,v in kw.items()}
    return [tag.lower(),c,kw]

In [ ]:
#| export
_g = globals()
_all_ = ['Html', 'Head', 'Title', 'Meta', 'Link', 'Style', 'Body', 'Pre', 'Code',
'Div', 'Span', 'P', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'Strong', 'Em', 'B',
'I', 'U', 'S', 'Strike', 'Sub', 'Sup', 'Hr', 'Br', 'Img', 'A', 'Link', 'Nav',
'Ul', 'Ol', 'Li', 'Dl', 'Dt', 'Dd', 'Table', 'Thead', 'Tbody', 'Tfoot', 'Tr',
'Th', 'Td', 'Caption', 'Col', 'Colgroup', 'Form', 'Input', 'Textarea',
'Button', 'Select', 'Option', 'Label', 'Fieldset', 'Legend', 'Details',
'Summary', 'Main', 'Header', 'Footer', 'Section', 'Article', 'Aside', 'Figure',
'Figcaption', 'Mark', 'Small', 'Iframe', 'Object', 'Embed', 'Param', 'Video',
'Audio', 'Source', 'Canvas', 'Svg', 'Math', 'Script', 'Noscript', 'Template', 'Slot']

for o in _all_: _g[o] = partial(xt, o.lower())

The main HTML tags are exported as `xt` partials.

Attributes are passed as keywords. Use 'klass' and 'fr' instead of 'class' and 'for', to avoid Python reserved word clashes.

In [ ]:
samp = Html(
    Head(Title('Some page')),
    Body(Div(P('Some text'), Img(src="filename"), klass='myclass'))
)
samp

['html',
 (['head', (['title', ('Some page',), {}],), {}],
  ['body',
   (['div',
     (['p', ('Some text',), {}], ['img', (), {'src': 'filename'}]),
     {'class': 'myclass'}],),
   {}]),
 {}]

In [ ]:
#| export
def to_xml(elm, lvl=0):
    "Convert `xt` element tree into an XML string"
    if hasattr(elm, '__xt__'): elm = elm.__xt__()
    sp = ' ' * lvl
    if not isinstance(elm, list):
        if isinstance(elm, str): elm = escape(elm)
        return f'{elm}\n'

    tag,cs,attrs = elm
    stag = tag
    if attrs:
        sattrs = (f'{k}="{escape(str(v))}"' for k,v in attrs.items())
        stag += ' ' + ' '.join(sattrs)
    
    if not cs: return f'{sp}<{stag}></{tag}>\n'
    res = f'{sp}<{stag}>\n'
    res += ''.join(to_xml(c, lvl+2) for c in cs)
    res += f'{sp}</{tag}>\n'
    return res

In [ ]:
Markdown(f'''```html
{to_xml(samp)}
```''')

```html
<html>
  <head>
    <title>
Some page
    </title>
  </head>
  <body>
    <div class="myclass">
      <p>
Some text
      </p>
      <img src="filename"></img>
    </div>
  </body>
</html>

```

# Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()